In [1]:
from imports import *
from functions import *
from constants import *
# Active l'extension autoreload dans un notebook Jupyter
%load_ext autoreload
%autoreload 2

In [2]:
def print_with_timestamp(message):
    now = dt.now().strftime("%Y-%m-%d %H:%M:%S")
    print(f"{now} - {message}")


# Launching Driver and Website Main Page

In [3]:
driver = create_driver()
driver.get(website_url)
destroy_intrusive_elements(driver)
wait = WebDriverWait(driver, timeout=3)
mi_cuenta = wait.until(EC.presence_of_element_located((By.XPATH, "//span[normalize-space()='Mi cuenta']")))  # Remplacer par l'élément attendu
mi_cuenta.click()
# Envoie de l'email sur le site
try:
    ingresar_email_zone = wait.until(EC.presence_of_element_located((By.XPATH, "//input[@placeholder='Ingresa tu email']")))
    ingresar_email_zone.clear()
    ingresar_email_zone.send_keys(EMAIL)
    ingresar_email_zone.send_keys(Keys.ENTER)
except:
    print('NO')

Waiting Function statement

In [4]:
wait10 = WebDriverWait(driver, timeout=10)

# Search Mechanic

In [5]:
SEARCH_QUERY = "portatil ASUS 1TB RTX RYZEN"

Dynamically Building the Name of the CSV File to store collected Data

In [6]:
csv_output_name = f"Exito_Search_{SEARCH_QUERY.replace(" ", "_")}.csv"
csv_output_name

'Exito_Search_portatil_ASUS_1TB_RTX_RYZEN.csv'

In [7]:
dict_results = {}
today = dt.now()
dict_results["Timestamp"] = today

In [18]:
driver.get(website_url)

try:
    # Attendre que la page soit complètement chargée
    wait10.until(lambda driver: driver.execute_script("return document.readyState") == "complete")
    print("Page completely loaded!")

    # Try to find the search field
    try:
        search_field = wait10.until(
            EC.element_to_be_clickable(
                (By.XPATH, "//input[@data-fs-input='true' and @aria-label='search']")
            )
        )
        print("Search field found and is clickable!")
    except Exception as e:
        print(f"❌ Error while finding search field: {e}")
        raise

    try:
        # Ensure the field is clear (use JavaScript to clear if necessary)
        driver.execute_script("arguments[0].value = '';", search_field)
        print("Field cleared using JavaScript!")
    except Exception as e:
        print(f"❌ Error while clearing field using JavaScript: {e}")
        raise

    try:
        # Optionally, use BACKSPACE to ensure the field is emptied
        current_value = search_field.get_attribute('value')
        if current_value:
            search_field.send_keys(Keys.BACKSPACE * len(current_value))
            print("Text cleared using BACKSPACE.")
        else:
            print("No text to clear with BACKSPACE.")
    except Exception as e:
        print(f"❌ Error while using BACKSPACE to clear text: {e}")
        raise

    try:
        # Send the search query
        search_field.send_keys(SEARCH_QUERY)
        search_field.send_keys(Keys.ENTER)
        print(f"Search query '{SEARCH_QUERY}' sent!")
    except Exception as e:
        print(f"❌ Error while sending search query: {e}")
        raise

except Exception as e:
    print(f"General error during search process: {str(e)}")

Page completely loaded!
Search field found and is clickable!
Field cleared using JavaScript!
No text to clear with BACKSPACE.
Search query 'portatil ASUS 1TB RTX RYZEN' sent!


In [ ]:
count = 0
page_num = 1

while True:
    ######################## GET PAGE ##########################################################################################################################################################################################
    wait10.until(lambda driver: driver.execute_script("return document.readyState") == "complete")
    print("Page completely loaded!")
    time_sleep(page=page_num, action="navigate", page_type="listing")
    print(f"🚀 Navigating to product listing page (Page {page_num})...")

    try:
        product_listing = wait10.until(EC.presence_of_element_located((By.XPATH, "//ul[contains(@data-fs-product-grid, 'true')]")))
        cards = product_listing.find_elements(By.XPATH, ".//div[contains(@class, 'productCard_contentInfo')]")
        total_results_str = wait10.until(EC.presence_of_element_located((By.XPATH, "//p[contains(@data-testid, 'total-product-count')]"))).text
        match = re.search(r"\d{1,9}", total_results_str)

        if match:
            total_results = int(match.group())
            dict_results["Total Result"] = total_results
            print(f"🔢 Total products found: {total_results}")

    except Exception as e:
        print(f"❌ Error while fetching the product listing: {e}")
        break

    ######################## LOOP ON CARDS ##########################################################################################################################################################################################

    for index in range(len(cards)):
        count += 1
        datas = {}

        try:
            # Recharger les cartes pour éviter stale
            product_listing = wait10.until(EC.presence_of_element_located((By.XPATH, "//ul[contains(@data-fs-product-grid, 'true')]")))
            cards = product_listing.find_elements(By.XPATH, ".//div[contains(@class, 'productCard_contentInfo')]")
            card = cards[index]
        except Exception as e:
            print(f"❌ Error refreshing card {index}: {e}")
            break

        ######################## GET TITLE & PRICE ##########################################################################################################################################################################################

        try:
            time_sleep(page=page_num, action="scroll", page_type="listing")
            print("📝 Scraping product title...")
            title = wait10.until(EC.presence_of_element_located((By.XPATH, "//h3[contains(@class, 'styles_name__qQJiK')]")))
            datas["Title"] = title.text
            print(f"📦 Product Title: {title.text}")

            actual_price = card.find_element(By.XPATH, ".//p[contains(@class, 'ProductPrice')]").text
            datas["Price"] = actual_price
            print(f"💰 Current Price: {actual_price}")

            ######################## GET DISCOUNT ##########################################################################################################################################################################################

            try:
                last_price = card.find_element(By.XPATH, ".//p[contains(@class, 'promotion_price')]").text
                datas["Last Price"] = last_price
                print(f"💸 Last Price: {last_price}")
                cleaned_actual_price = actual_price.replace('$', '').replace(',', '').strip()
                cleaned_last_price = last_price.replace('$', '').replace(',', '').strip()
                discount_value = get_discount_percentage(cleaned_last_price, cleaned_actual_price)
                datas["Discount"] = discount_value
                print(f"🔻 Discount: {discount_value}%")
            except NoSuchElementException:
                print("❌ No promotion available for this product.")

            ######################## GET IMAGES ##########################################################################################################################################################################################

            try:
                images_zone = wait10.until(EC.presence_of_element_located((By.XPATH, ".//img[contains(@alt, 'Imagen del producto')]")))
                img_url = images_zone.get_attribute('src')
                datas["Original Image"] = img_url
                print(f"🖼️ Original Image URL: {img_url}")
                ActionChains(driver).move_to_element(images_zone).perform()
                new_img_url = images_zone.get_attribute('src')
                if new_img_url != img_url:
                    datas["Hovered Image"] = new_img_url
                    print(f"🔄 Hovered Image URL: {new_img_url}")
                else:
                    print("🖼️ No change in image after hover.")
            except NoSuchElementException:
                print("❌ Image not found.")

            ######################## PRODUCT DETAIL PAGE ##########################################################################################################################################################################################

            print("🔗 Navigating to product detail page...")
            link_producto_detalles = find_element(card, "xpath", ".//a[contains(@data-testid,'product-link')]").get_attribute('href')
            time_sleep(page=page_num, action="click", page_type="listing")
            driver.get(link_producto_detalles)
            time_sleep(page=page_num, action="navigate", page_type="product")
            product_section = wait10.until(EC.presence_of_element_located((By.XPATH, "//section[contains(@class, 'section product-details')]")))

            ######################## PLU ID ##########################################################################################################################################################################################

            PLU_ID = find_element(product_section, "xpath", ".//span[contains(@class, 'product-title__specification')]").text
            datas["PLU_ID"] = PLU_ID
            print(f"🆔 PLU ID: {PLU_ID}")

            ######################## FULL IMAGES ##########################################################################################################################################################################################

            imgs_tags = find_elements(product_section, "xpath", ".//img[contains(@src, '')]")
            imgs_url = [img.get_attribute('src') for img in imgs_tags]
            imgs_url = filter_products_img_urls(imgs_url)
            datas['Images URL'] = imgs_url
            print(f"🖼️ Product Images: {imgs_url}")

            ######################## STOCK ##########################################################################################################################################################################################

            stock_details = find_element(product_section, "xpath", ".//p[@data-fs-product-details-stock__qty='true']").text
            datas["Stock Details"] = stock_details
            print(f"📦 Stock Status: {stock_details}")

            ######################## DEALER ##########################################################################################################################################################################################

            dealer_part = find_element(product_section, "xpath", ".//a[contains(@href, 'seller')]")
            dealer = dealer_part.text
            datas["Dealer"] = dealer
            dealer_url = dealer_part.get_attribute("href")
            datas["Dealer URL"] = dealer_url
            print(f"🏢 Dealer: {dealer}, URL: {dealer_url}")

            ######################## PRODUCT SPECIFICATIONS ##########################################################################################################################################################################################

            try:
                article_text_container = find_element(product_section, "xpath", ".//article[contains(@data-accordion-item, 'true') and contains(@data-testid, 'store-accordion-item-item')]")
                product_specifications = find_element(article_text_container, "xpath", ".//div[contains(@data-fs-content-specification,'true')]")
                detalles = find_elements(product_specifications, "xpath", ".//div[contains(@data-fs-specification,'')]")
                
                for div in detalles:
                    try:
                        paragraph_title = find_element(div, "xpath", ".//p[contains(@data-fs-title-specification, 'true')]").text
                        paragraph_text = find_element(div, "xpath", ".//p[contains(@data-fs-text-specification, 'true')]").text
                        detail = f"{paragraph_title}: {paragraph_text}"
                        print(f"📋 Product Specification: {detail}")
                    except (NoSuchElementException, TimeoutException):
                        pass
            except (NoSuchElementException, TimeoutException):
                print("🔍 Could not find the main product specification section.")


            ######################## PRODUCT DESCRIPTION ##########################################################################################################################################################################################

            product_description = find_element(article_text_container, "xpath", ".//div[contains(@data-fs-description-text, '')]").text
            datas["Product Description"] = product_description
            print(f"📝 Product Description: {product_description}")

            ######################## DELIVERY DETAILS ##########################################################################################################################################################################################

            delivery_details_zone = find_element(product_section, "xpath", ".//section[contains(@class, 'deliveryPromises')]")
            delivery_details = find_element(delivery_details_zone, "xpath", ".//span[contains(@data-fs-promises-bold,'true')]").text
            disponibilidad_compra_y_recoge = find_element(delivery_details_zone, "xpath", ".//p[contains(@data-fs-promises-bold,'true')]").text
            datas["Delivery Details"] = delivery_details
            datas["Purchase & Pick-up"] = disponibilidad_compra_y_recoge
            print(f"🚚 Delivery Info: {delivery_details}")
            print(f"🏪 Availability for Pick-up: {disponibilidad_compra_y_recoge}")

            ######################## SAVE & GO BACK ##########################################################################################################################################################################################

            dict_results[count] = datas

            driver.back()
            time_sleep(page=page_num, action="navigate", page_type="listing")

        except Exception as e:
            print(f"❌ Error occurred while scraping product: {e}")
            break

    ######################## NEXT PAGE ##########################################################################################################################################################################################

    try:
        time_sleep(page=page_num, action="click", page_type="listing")
        button_next_page = wait10.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'Pagination_next')]")))
        button_next_page.click()
        page_num += 1
        print(f"📄 Moving to page {page_num}...")
    except NoSuchElementException:
        print("🚫 No next page found. Stopping the loop...")
        break


# Data Collection

In [ ]:
count = 0
page_num = 1

######## LOOP ON PAGES #############################################################################################################################################################

while True:

    ######## GO TO LISTING PAGE #############################################################################################################################################################

    time_sleep(page=page_num, action="navigate", page_type="listing")
    print(f"🚀 Navigating to product listing page (Page {page_num})...")

    try:
        product_listing = wait10.until(EC.presence_of_element_located((By.XPATH, "//ul[contains(@data-fs-product-grid, 'true')]")))
        cards = product_listing.find_elements(By.XPATH, ".//div[contains(@class, 'productCard_contentInfo')]")

        total_results_str = wait10.until(EC.presence_of_element_located((By.XPATH, "//p[contains(@data-testid, 'total-product-count')]"))).text
        match = re.search(r"\d{1,9}", total_results_str)

        if match:
            total_results = int(match.group())
            dict_results["Total Result"] = total_results
            print(f"🔢 Total products found: {total_results}")

    except Exception as e:
        print(f"❌ Error while fetching the product listing: {e}")
        break

    ######## LOOP ON CARDS #############################################################################################################################################################

    for index in range(len(cards)):
        try:
            ######## REFRESH CARD #############################################################################################################################################################

            product_listing = wait10.until(EC.presence_of_element_located((By.XPATH, "//ul[contains(@data-fs-product-grid, 'true')]")))
            cards = product_listing.find_elements(By.XPATH, ".//div[contains(@class, 'productCard_contentInfo')]")
            card = cards[index]

            count += 1
            datas = {}

            ######## TITLE #############################################################################################################################################################

            time_sleep(page=page_num, action="scroll", page_type="listing")
            print("📝 Scraping product title...")

            title = wait10.until(EC.presence_of_element_located((By.XPATH, "//h3[contains(@class, 'styles_name__qQJiK')]")))
            datas["Title"] = title.text
            print(f"📦 Product Title: {title.text}")

            ######## PRICE #############################################################################################################################################################

            actual_price = card.find_element(By.XPATH, ".//p[contains(@class, 'ProductPrice')]").text
            datas["Price"] = actual_price
            print(f"💰 Current Price: {actual_price}")

            try:
                last_price = card.find_element(By.XPATH, ".//p[contains(@class, 'promotion_price')]").text
                datas["Last Price"] = last_price
                print(f"💸 Last Price: {last_price}")

                cleaned_actual_price = actual_price.replace('$', '').replace(',', '').strip()
                cleaned_last_price = last_price.replace('$', '').replace(',', '').strip()
                discount_value = get_discount_percentage(cleaned_last_price, cleaned_actual_price)
                datas["Discount"] = discount_value
                print(f"🔻 Discount: {discount_value}%")
            except NoSuchElementException:
                print("❌ No promotion available for this product.")

            ######## IMAGES #############################################################################################################################################################

            try:
                images_zone = wait10.until(EC.presence_of_element_located((By.XPATH, ".//img[contains(@alt, 'Imagen del producto')]")))
                img_url = images_zone.get_attribute('src')
                datas["Original Image"] = img_url
                print(f"🖼️ Original Image URL: {img_url}")

                ActionChains(driver).move_to_element(images_zone).perform()
                new_img_url = images_zone.get_attribute('src')

                if new_img_url != img_url:
                    datas["Hovered Image"] = new_img_url
                    print(f"🔄 Hovered Image URL: {new_img_url}")
                else:
                    print("🖼️ No change in image after hover.")

            except NoSuchElementException:
                print("❌ Image not found.")

            ######## PRODUCT DETAIL PAGE #############################################################################################################################################################

            print("🔗 Navigating to product detail page...")
            link_producto_detalles = find_element(card, "xpath", ".//a[contains(@data-testid,'product-link')]").get_attribute('href')
            time_sleep(page=page_num, action="click", page_type="listing")
            driver.get(link_producto_detalles)
            time_sleep(page=page_num, action="navigate", page_type="product")

            product_section = wait10.until(EC.presence_of_element_located((By.XPATH, "//section[contains(@class, 'section product-details')]")))

            ######## PLU ID #############################################################################################################################################################

            PLU_ID = find_element(product_section, "xpath", ".//span[contains(@class, 'product-title__specification')]").text
            datas["PLU_ID"] = PLU_ID
            print(f"🆔 PLU ID: {PLU_ID}")

            ######## PRODUCT IMAGES #############################################################################################################################################################

            imgs_tags = find_elements(product_section, "xpath", ".//img[contains(@src, '')]")
            imgs_url = [img.get_attribute('src') for img in imgs_tags]
            imgs_url = filter_products_img_urls(imgs_url)
            datas['Images URL'] = imgs_url
            print(f"🖼️ Product Images: {imgs_url}")

            ######## STOCK #############################################################################################################################################################

            stock_details = find_element(product_section, "xpath", ".//p[@data-fs-product-details-stock__qty='true']").text
            datas["Stock Details"] = stock_details
            print(f"📦 Stock Status: {stock_details}")

            ######## DEALER #############################################################################################################################################################

            dealer_part = find_element(product_section, "xpath", ".//a[contains(@href, 'seller')]")
            dealer = dealer_part.text
            datas["Dealer"] = dealer
            dealer_url = dealer_part.get_attribute("href")
            datas["Dealer URL"] = dealer_url
            print(f"🏢 Dealer: {dealer}, URL: {dealer_url}")

            ######## PRODUCT SPECIFICATIONS #############################################################################################################################################################

            article_text_container = find_element(product_section, "xpath", ".//article[contains(@data-accordion-item, 'true') and contains(@data-testid, 'store-accordion-item-item')]")
            product_specifications = find_element(article_text_container, "xpath", ".//div[contains(@data-fs-content-specification,'true')]")
            detalles = find_elements(product_specifications, "xpath", ".//div[contains(@data-fs-specification,'')]")

            for div in detalles:
                paragraph_title = find_element(div, "xpath", ".//p[contains(@data-fs-title-specification, 'true')]").text
                paragraph_text = find_element(div, "xpath", ".//p[contains(@data-fs-text-specification, 'true')]").text
                detail = f"{paragraph_title}: {paragraph_text}"
                print(f"📋 Product Specification: {detail}")

            ######## PRODUCT DESCRIPTION #############################################################################################################################################################

            product_description = find_element(article_text_container, "xpath", ".//div[contains(@data-fs-description-text, '')]").text
            datas["Product Description"] = product_description
            print(f"📝 Product Description: {product_description}")

            ######## DELIVERY #############################################################################################################################################################

            delivery_details_zone = find_element(product_section, "xpath", ".//section[contains(@class, 'deliveryPromises')]")
            delivery_details = find_element(delivery_details_zone, "xpath", ".//span[contains(@data-fs-promises-bold,'true')]").text
            disponibilidad_compra_y_recoge = find_element(delivery_details_zone, "xpath", ".//p[contains(@data-fs-promises-bold,'true')]").text
            datas["Delivery Details"] = delivery_details
            datas["Purchase & Pick-up"] = disponibilidad_compra_y_recoge
            print(f"🚚 Delivery Info: {delivery_details}")
            print(f"🏪 Availability for Pick-up: {disponibilidad_compra_y_recoge}")

            ######## STORE RESULTS #############################################################################################################################################################

            dict_results[count] = datas

            ######## BACK TO LISTING #############################################################################################################################################################

            driver.back()
            time_sleep(page=page_num, action="navigate", page_type="listing")
            wait10.until(EC.presence_of_element_located((By.XPATH, "//ul[contains(@data-fs-product-grid, 'true')]")))

        except Exception as e:
            print(f"❌ Error occurred while scraping product: {e}")
            break

    ######## NEXT PAGE #############################################################################################################################################################

    try:
        time_sleep(page=page_num, action="click", page_type="listing")
        button_next_page = wait10.until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'Pagination_next')]")))
        button_next_page.click()
        page_num += 1
        print(f"📄 Moving to page {page_num}...")
    except NoSuchElementException:
        print("🚫 No next page found. Stopping the loop...")
        break


In [ ]:
total_results_str = wait10.until(EC.presence_of_element_located((By.XPATH, "//p[contains(@data-testid, 'total-product-count')]"))).text
match = re.search(r"\d{1,9}", total_results_str)

if match:
    total_results = int(match.group())
    print(f"Total Result : {total_results}")

In [ ]:
driver.get("https://www.exito.com/s?q=portatil+ASUS+Ryzen+7+1TB+RAM&sort=score_desc&page=0")

product_listing = wait10.until(EC.presence_of_element_located((By.XPATH, "//ul[contains(@data-fs-product-grid, 'true')]")))
cards = product_listing.find_elements(By.XPATH, ".//div[contains(@class, 'productCard_contentInfo')]")

one_card = cards[0]

# Article Details

In [ ]:
############################ PRODUCT PAGE SCRAPING ###############################################################################################################################################
##################################################################################################################################################################################################

time_sleep(page=page_num, action="navigate", page_type="product")
product_section = wait10.until(EC.presence_of_element_located((By.XPATH, "//section[contains(@class, 'section product-details')]")))

######################### GETTING PRODUCT ID ###############################################################################################################################################

PLU_ID = find_element(product_section, "xpath", ".//span[contains(@class, 'product-title__specification')]").text
datas["PLU_ID"] = PLU_ID
print(f"🆔 PLU ID: {PLU_ID}")

######################### GET PRODUCT IMAGES ###############################################################################################################################################

imgs_tags = find_elements(product_section, "xpath", ".//img[contains(@src, '')]")
imgs_url = [img.get_attribute('src') for img in imgs_tags]
imgs_url = filter_products_img_urls(imgs_url)
datas['Images URL'] = imgs_url
print(f"🖼️ Product Images: {imgs_url}")

######################### GETTING STOCK DETAILS ###############################################################################################################################################

stock_details = find_element(product_section, "xpath", ".//p[@data-fs-product-details-stock__qty='true']").text
datas["Stock Details"] = stock_details
print(f"📦 Stock Status: {stock_details}")

######################### GETTING DEALER DETAILS ###############################################################################################################################################

dealer = find_element(product_section, "xpath", ".//a[contains(@href, 'seller')]").text

datas["Dealer"] = dealer
dealer_url = dealer_part.get_attribute("href")
datas["Dealer URL"] = dealer_url
print(f"🏢 Dealer: {dealer}, URL: {dealer_url}")

######################### GETTING PRODUCT SPECIFICATIONS ###############################################################################################################################################

article_text_container = find_element(product_section, "xpath", ".//article[contains(@data-accordion-item, 'true') and contains(@data-testid, 'store-accordion-item-item')]")
product_specifications = find_element(article_text_container, "xpath", ".//div[contains(@data-fs-content-specification,'true')]")
detalles = find_elements(product_specifications, "xpath", ".//div[contains(@data-fs-specification,'')]")
for div in detalles:
    paragraph_title = find_element(div, "xpath", ".//p[contains(@data-fs-title-specification, 'true')]").text
    paragraph_text = find_element(div, "xpath", ".//p[contains(@data-fs-text-specification, 'true')]").text
    detail = f"{paragraph_title}: {paragraph_text}"
    print(f"📋 Product Specification: {detail}")

######################### GETTING PRODUCT DESCRIPTION ###############################################################################################################################################

product_description = find_element(article_text_container, "xpath", ".//div[contains(@data-fs-description-text, '')]").text
datas["Product Description"] = product_description
print(f"📝 Product Description: {product_description}")

delivery_details_zone = find_element(product_section, "xpath", ".//section[contains(@class, 'deliveryPromises')]")
delivery_details = find_element(delivery_details_zone, "xpath", ".//span[contains(@data-fs-promises-bold,'true')]").text
disponibilidad_compra_y_recoge = find_element(delivery_details_zone, "xpath", ".//p[contains(@data-fs-promises-bold,'true')]").text
datas["Delivery Details"] = delivery_details
datas["Purchase & Pick-up"] = disponibilidad_compra_y_recoge
print(f"🚚 Delivery Info: {delivery_details}")
print(f"🏪 Availability for Pick-up: {disponibilidad_compra_y_recoge}")

In [ ]:
##########  PRODUCT PAGE SCRAPING ###########################################################################################################################################################################################################################

product_section = find_element(driver, "xpath", "//section[contains(@class, 'section product-details')]")

PLU_ID = find_element(product_section, "xpath", ".//span[contains(@class, 'product-title__specification')]").text

# Récupération des images
imgs_tags = find_elements(product_section, "xpath", ".//img[contains(@src, '')]")

imgs_url = [img.get_attribute('src') for img in imgs_tags]

imgs_url =filter_products_img_urls(imgs_url)
print(imgs_url)

stock_details = find_element(product_section, "xpath", ".//p[@data-fs-product-details-stock__qty='true']").text

# Affiche le texte
print(f"Texte complet : {stock_details}")


# Trouver l'élément <a> contenant "Mundo digital vm"
dealer_part = find_element(product_section, "xpath", ".//a[contains(text(), '')]")

# Récupérer le texte de l'élément <a> (le texte "Mundo digital vm")
dealer = dealer_part.text
print(f"Dealer: {dealer}")

# Récupérer l'URL associée à cet élément <a>
dealer_url = dealer_part.get_attribute("href")
print(f"Dealer URL : {dealer_url}")


article_text_container = find_element(product_section, 
    "xpath", ".//article[contains(@data-accordion-item, 'true') and contains(@data-testid, 'store-accordion-item-item')]")

product_specifications = find_element(article_text_container, 
            "xpath", ".//div[contains(@data-fs-content-specification,'true')]")
detalles = find_elements(product_specifications, 
        "xpath", ".//div[contains(@data-fs-specification,'')]")
len(detalles)
for div in detalles:
    paragraph_title = find_element(div, "xpath", ".//p[contains(@data-fs-title-specification, 'true')]").text
    paragraph_text = find_element(div, "xpath", ".//p[contains(@data-fs-text-specification, 'true')]").text
    detail = f"{paragraph_title}: {paragraph_text}"
    print(detail)
product_description = find_element(article_text_container, "xpath", 
                    ".//div[contains(@data-fs-description-text, '')]").text

normalized_description = normalize_text(product_description)
print(normalized_description)

delivery_details_zone = find_element(product_section, "xpath", 
".//section[contains(@class, 'deliveryPromises')]")
delivery_details = find_element(delivery_details_zone, "xpath",
            ".//span[contains(@data-fs-promises-bold,'true')]").text
disponibilidad_compra_y_recoge = find_element(delivery_details_zone, "xpath",
            ".//p[contains(@data-fs-promises-bold,'true')]").text

##########  ###########################################################################################################################################################################################################################



Go on the Computer Page

In [ ]:
category = "Tecnología"
article_type_choice = "Portátiles"

main_page=driver.current_url
def go_to_article_page(driver, category, article_type):
    import time
    from selenium.webdriver.common.by import By

    try:
        # Récupère le menu des catégories
        categories_menu = driver.find_element(By.XPATH, '//*[@id="header-page"]/aside/ul/section[2]')

        # Cherche tous les div ayant l'attribut data-link-content="true"
        divs_with_data_link_content = categories_menu.find_elements(By.XPATH, './/div[@data-link-content="true"]')

        # Clique sur la catégorie principale
        for div in divs_with_data_link_content:
            if div.text.strip() == category:
                div.click()
                print(f"Catégorie '{category}' cliquée.")
                time.sleep(1)
                break

        # Menu déroulé : chercher le type d'article
        selected_menu = driver.find_element(By.XPATH, './/ul[@data-content-list="true"]')
        list_elements = selected_menu.find_elements(By.TAG_NAME, 'li')

        for li in list_elements:
            if li.text.strip() == article_type:
                try:
                    link = li.find_element(By.TAG_NAME, 'a')
                    link.click()
                    print(f"Lien cliqué vers la page '{article_type}'")
                    time.sleep(2)
                    return driver.current_url
                except:
                    print(f"Lien introuvable dans l'élément '{article_type}'")
                    return None
        print(f"Type d'article '{article_type}' non trouvé.")
        return None

    except Exception as e:
        print(f"Erreur lors de la navigation : {e}")
        return None
try:
    go_to_article_page(driver, category, article_type_choice)
except Exception as e:
    print(str(e))

Getting all the Checkboxes

In [ ]:
computer_filter_menu = driver.find_element(By.XPATH, "//div[@class='accordion_fs-accordion__mi4MR']")

# Récupère tous les titres des filtres (balises h4)
filters = computer_filter_menu.find_elements(By.TAG_NAME, 'h4')
Sub_categoría = filters[1]

# desktop-store-filter-Sub-categoría-Accesorios de cámaras

# Vérifier si l'élément Sub_categoría a été trouvé
if Sub_categoría:
    # Trouver toutes les checkboxes à l'intérieur de 'Sub_categoría'
    checkboxes = Sub_categoría.find_elements(By.XPATH, '//input[@type="checkbox" and @data-fs-checkbox="true"]')
    print(f"Nombre de checkboxes trouvées : {len(checkboxes)}")
    for checkbox in checkboxes:
        print(checkbox.get_attribute("data-value"))
else:
    print("L'élément 'Sub-categoría' n'a pas été trouvé.")

In [ ]:
def cocher_checkboxes(filters_choice, checkboxes, driver):
    # Normalisation des filtres
    filters_normalized = [unidecode(item.lower()) for item in filters_choice]

    for i in range(len(checkboxes)):
        try:
            # On relocalise à chaque boucle pour éviter stale
            checkboxes_refreshed = driver.find_elements(By.CSS_SELECTOR, "input[type='checkbox'][data-value]")
            if i >= len(checkboxes_refreshed):
                break  # Au cas où le DOM change

            checkbox = checkboxes_refreshed[i]

            data_value_raw = checkbox.get_attribute("data-value")
            if data_value_raw is None:
                continue

            data_value = unidecode(data_value_raw.replace("-", " ").lower())
            matched = any(f in data_value for f in filters_normalized)

            if matched and not checkbox.is_selected():
                click_with_JS(checkbox, driver)
                print(f"[CHECK] Checkbox pour '{data_value}' cochée.")

            elif not matched and checkbox.is_selected():
                click_with_JS(checkbox, driver)
                print(f"[UNCHECK] Checkbox pour '{data_value}' décochée.")
        
        except Exception as e:
            print(f"[ERROR] Problème sur checkbox {i}: {e}")
            continue

from unidecode import unidecode


In [ ]:
queries = [
    "asus", "8 gb", "16 GB", "nvidia", "amd", "24 gb", "32 gb", "ssd",
    "ryzen 7", "500 GB", "1 TB", "i7", "gaming", 'portatiles'
]

In [ ]:
try:
    cocher_checkboxes(filters_choice=queries, checkboxes=checkboxes,driver=driver)
except Exception as e:
    print(str(e))

In [ ]:
from currencies import Currency

class CurrencyConverter:
    def __init__(self, from_currency, to_currency, exchange_rate):
        self.from_currency = Currency(from_currency)
        self.to_currency = Currency(to_currency)
        self.exchange_rate = exchange_rate

    def convert(self, amount):
        # Effectuer la conversion de l'euro vers le COP
        amount_in_target_currency = amount * self.exchange_rate
        # Retourne un float sans formatage en chaîne
        return amount_in_target_currency

# Exemple d'utilisation :
# Taux de conversion (peut être récupéré depuis une API, ici 1 EUR = 4500 COP)
exchange_rate_eur_to_cop = 4500

# Création d'un convertisseur de devise
converter_eur_cop = CurrencyConverter('EUR', 'COP', exchange_rate_eur_to_cop)


In [ ]:
# Exemple d'utilisation
min_value = converter_eur_cop.convert(300)  # Conversion de 300 EUR
max_value = converter_eur_cop.convert(500)  # Conversion de 450 EUR

In [ ]:
def set_slider_value(driver, min_value, max_value):
    # Récupérer la valeur actuelle des deux sliders avant de les changer
    initial_left_value = driver.execute_script("""
    const leftSlider = document.querySelector('input[type="range"][data-slider-thumb="left"]');
    return leftSlider ? leftSlider.value : null;
    """)
    
    initial_right_value = driver.execute_script("""
    const rightSlider = document.querySelector('input[type="range"][data-slider-thumb="right"]');
    return rightSlider ? rightSlider.value : null;
    """)

    print(f"Valeur initiale du slider gauche : {initial_left_value}")
    print(f"Valeur initiale du slider droit : {initial_right_value}")

    # Changer la valeur des sliders via JavaScript
    script = """
    const leftSlider = document.querySelector('input[type="range"][data-slider-thumb="left"]');
    const rightSlider = document.querySelector('input[type="range"][data-slider-thumb="right"]');
    
    if (leftSlider) {
        leftSlider.value = arguments[0]; // Définir la nouvelle valeur du slider gauche
        leftSlider.dispatchEvent(new Event('input', { bubbles: true })); // Simuler l'événement 'input'
        leftSlider.dispatchEvent(new Event('change', { bubbles: true })); // Simuler l'événement 'change'
    }

    if (rightSlider) {
        rightSlider.value = arguments[1]; // Définir la nouvelle valeur du slider droit
        rightSlider.dispatchEvent(new Event('input', { bubbles: true })); // Simuler l'événement 'input'
        rightSlider.dispatchEvent(new Event('change', { bubbles: true })); // Simuler l'événement 'change'
    }
    """
    # Exécuter le script pour changer les valeurs des deux sliders
    driver.execute_script(script, min_value, max_value)

    # Récupérer la nouvelle valeur des deux sliders après modification
    final_left_value = driver.execute_script("""
    const leftSlider = document.querySelector('input[type="range"][data-slider-thumb="left"]');
    return leftSlider ? leftSlider.value : null;
    """)

    final_right_value = driver.execute_script("""
    const rightSlider = document.querySelector('input[type="range"][data-slider-thumb="right"]');
    return rightSlider ? rightSlider.value : null;
    """)

    print(f"Valeur après modification du slider gauche : {final_left_value}")
    print(f"Valeur après modification du slider droit : {final_right_value}")

    # Comparer les valeurs avant et après
    if initial_left_value != final_left_value:
        print(f"Le slider gauche a été mis à jour avec succès. Nouvelle valeur : {final_left_value}")
    else:
        print(f"Aucune modification n'a été effectuée sur le slider gauche. Valeur actuelle : {final_left_value}")

    if initial_right_value != final_right_value:
        print(f"Le slider droit a été mis à jour avec succès. Nouvelle valeur : {final_right_value}")
    else:
        print(f"Aucune modification n'a été effectuée sur le slider droit. Valeur actuelle : {final_right_value}")


In [ ]:
set_slider_value(driver, min_value, max_value)

In [ ]:
aplicar_filtros = driver.find_element(By.XPATH, "//button[normalize-space()='Aplicar filtros']")
aplicar_filtros.click()